In [1]:
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q datasets
!pip install -q gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.7 MB/s eta 0:00:00


In [2]:
from transformers import XGLMTokenizer, XGLMForCausalLM, GenerationConfig
import torch
def load_model(base_model:str="ohmreborn/xglm-564M"):
    tokenizer = XGLMTokenizer.from_pretrained(base_model)
    model = XGLMForCausalLM.from_pretrained(base_model,torch_dtype=torch.float16)
    return model,tokenizer
model,tokenizer = load_model()

In [3]:
import torch
generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
    )
device = torch.device('cuda')
model.to(device)

XGLMForCausalLM(
  (model): XGLMModel(
    (embed_tokens): Embedding(256010, 1024, padding_idx=1)
    (embed_positions): XGLMSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-23): 24 x XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine

In [4]:
import re
def gen(prompt,generation_config=generation_config):
  prompt = f'{prompt} <human>:'
  inputs = tokenizer(prompt, return_tensors="pt")
  input_ids = inputs["input_ids"]
  input_ids = input_ids.to(device)
  def get_res(inputs):
    with torch.no_grad():
      generation_output = model.generate(
          input_ids=input_ids,
          generation_config=generation_config,
          return_dict_in_generate=True,
          output_scores=True,
          max_new_tokens=256,
      )
    output_1_ids = generation_output.sequences[0]
    output = tokenizer.decode(output_1_ids)  
    return output
  output = get_res(inputs)
  if '<bot>:' in output:
    res = re.split('<human>:|<bot>:|</s>',output)
    return f'<human>: {res[2]} \n<bot>: {res[3]}'
  else:
    bot_prompt = re.split(tokenizer.eos_token,output)[1]
    bot_prompt = f'{bot_prompt} <bot>:'

    inputs = tokenizer(bot_prompt, return_tensors="pt")
    input_ids = inputs["input_ids"]
    input_ids = input_ids.to(device)
    output = get_res(inputs)
    res = re.split('<human>:|<bot>:|</s>',output)
    return f'<human>: {res[2]} \n<bot>: {res[2]}'

In [5]:
sample = """
Doraemon  is a Japanese manga series written and illustrated 
by Fujiko F. Fujio. The manga was first serialized in December 1969, 
with its 1,345 individual chapters compiled into 45 tankōbon volumes 
and published by Shogakukan from 1970 to 1996. The story revolves around 
an earless robotic cat named Doraemon, who travels back in time from the 
22nd century to aid a boy named Nobita Nobi."""

In [6]:
response = gen(sample)
print(response)

<human>:  What is Doraemon  
<bot>:  Doraemon Doraemon is a Japanese manga series written and illustrated by Fujiko F. Fujio


#ลอง gen data ของ เจน

In [7]:
import gdown
url = "https://drive.google.com/uc?export=download&id=1ugUZYFBO7UyC11Fce8atVD_sdxXoJ7fE"
output = 'data.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1ugUZYFBO7UyC11Fce8atVD_sdxXoJ7fE
To: /content/data.csv
100%|██████████| 28.9M/28.9M [00:00<00:00, 157MB/s] 


'data.csv'

In [8]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,title,description,tags,cover
0,0,Salad Days (Tang LiuZang) - Part 2,The second season of Salad Days (Tang LiuZang).,"['BL', 'Manhua', 'Romance', 'Shounen-ai', 'Spo...",https://cdn.anime-planet.com/manga/primary/sal...
1,1,The Master of Diabolism,As the grandmaster who founded the Demonic Sec...,"['Action', 'Adventure', 'BL', 'Comedy', 'Manhu...",https://cdn.anime-planet.com/manga/primary/the...
2,2,JoJo's Bizarre Adventure Part 7: Steel Ball Run,"Set in 1890, Steel Ball Run spotlights Gyro Ze...","['Action', 'Adventure', 'Horror', 'Mystery', '...",https://cdn.anime-planet.com/manga/primary/joj...
3,3,A Sign of Affection,"Yuki is a typical college student, whose world...","['Romance', 'Shoujo', 'Slice of Life', 'Disabi...",https://cdn.anime-planet.com/manga/primary/a-s...
4,4,Moriarty the Patriot,"Before he was Sherlock’s rival, Moriarty fough...","['Mystery', 'Shounen', 'Detectives', 'England'...",https://cdn.anime-planet.com/manga/primary/mor...


In [9]:
# df[df['title'].str.startswith('z')]

In [10]:
df.iloc[451,2]

'While working as a small supporting actor in a movie, Ki Man Sung has nothing else going for him. However, that was when, through mysterious circumstance, the movie set had become reality during filming. With the sound of the director’s cut, during that short take, he’s not an actor but actually thrown into the events occurring, showing the pain and emotions he faces in that moment. He can only come back if he creates that perfect take…! Can he grow to become the greatest actor?'

In [11]:
gen(df.iloc[451,2])

'<human>:  Provide one sentence about Ki Man Sung  \n<bot>:  While working as a small supporting actor in a movie'